## Streaming

<img src="./assets/LC_streaming.png" width="400">

스트리밍은 데이터 생성 시점과 사용자가 이를 수신하는 시점 사이의 지연 시간을 줄입니다.
에이전트와 함께 자주 사용되는 두 가지 유형이 있습니다:

In [1]:
from dotenv import load_dotenv
from env_utils import doublecheck_env

# Load environment variables from .env
load_dotenv("../.env")

# Check and print results
doublecheck_env("../.env")

OPENAI_API_KEY=****VLwA
UPSTAGE_API_KEY=****d51g
LANGCHAIN_TRACING_V2=false
LANGCHAIN_ENDPOINT=****.com
LANGCHAIN_PROJECT=****demy
LANGCHAIN_API_KEY=****fc10
PINECONE_API_KEY=****4e73
HUGGINGFACE_API_KEY=****aRMB
TAVILY_API_KEY=****2UUC
SERPER_API_KEY=****e4cd
WOLFRAM_ALPHA_APPID=****LT74
POLYGON_API_KEY=****8Gso


In [2]:
from langchain.agents import create_agent

In [4]:
agent = create_agent(
    model="openai:gpt-5",
    system_prompt="당신은 풀스택 코미디언입니다.",
)

## No Steaming (invoke)

In [5]:
result = agent.invoke({"messages": [{"role": "user", "content": "나에게 농담을 건네보세요."}]})
print(result["messages"][1].content)

- 요즘 내 연애는 404야. 상대가 Found가 안 되네.
- 썸에도 API가 있어. 너무 자주 호출하면 429: 잠깐만 쉬자.
- 헤어지고 커밋 남겼다: fix: relationship bug. 결과? merge conflict.
- 사랑은 비공개 레포지토리. 신뢰 키 없으면 접근 불가.
- 인생 문제 스택오버플로우에 물었더니 최고답변: 물 마시고 자.
- 운동 2일 하다 배운 교훈: CI는 했지만 CD는 못 하겠다.
- 카페에서 깨달음: 내 커피 취향은 철저한 다크 모드.
- 나는 풀스택 코미디언. 앞에서는 웃기고, 뒤에서는 왜 웃었는지 로그 남긴다.


## Streaming
스트리밍은 values 스트림, messages 스트림 두가지 모드가 있다.

## values
지금까지 예제에서 이 스트리밍 모드를 보셨을 것입니다. 

In [6]:
# Stream = values
for step in agent.stream(
    {"messages": [{"role": "user", "content": "Tell me a Dad joke"}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Tell me a Dad joke
================================== Ai Message ==================================

I ordered a chicken and an egg from Amazon—I’ll let you know which comes first.


## messages
메시지는 토큰 단위로 데이터를 스트리밍합니다 - 가능한 최저 지연 시간을 제공합니다. 이는 챗봇과 같은 대화형 애플리케이션에 완벽합니다.

In [8]:
for token, metadata in agent.stream(
    {"messages": [{"role": "user", "content": "가족 모두가 즐길 수 있는 시를 써 주세요."}]},
    stream_mode="messages",
):
    print(f"{token.content}", end="")

가족 사용설명서 (농담 포함)

아침엔 알람이 먼저 울리지만, 진짜 관리자 계정은 밥 냄새.
숟가락은 로그인 키, 오늘의 국은 따끈한 업데이트.

아빠의 농담은 자동 업데이트, 롤백은 미지원.
엄마는 한 손으로 국 젓고, 다른 손으로 분쟁 해결(장난감 분쟁 전문 조정사).

할머니의 쿠키는 레시피보다 마음이 많다—비밀 재료는 “조금 더”.
할아버지는 리모컨 묘기단: 채널도 돌리고, 허리도 살짝 돌리고.

빨래 바구니엔 미스터리 사건: 양말은 왜 자꾸 솔로 데뷔를 하나?
신발장 앞에서는 매일 공연: “한 짝 찾습니다!” 앵콜 확정.

동생의 연필은 드럼 스틱, 형의 숙제는 리듬에 맞춰 흔들.
누나의 웃음은 번역가—투덜거림도 해석하면 “같이 놀자”.

현관 벨 소리는 우리 집 배경음악, “띵동!” 하면 모두가 주연.
식탁 위 회의의 결론은 늘 같다: 의제 1번, 맛있다. 가결.

고양이(혹은 강아지)는 품질보증팀: 떨어뜨려 보고, 눕혀 보고, 합격이면 꾹꾹이 도장.
창밖 별은 야간 조명, 꿈은 서로의 마음 서버에 자동 백업.

우리 집은 풀스택 가족:
웃음이 프론트엔드, 배려가 백엔드, 사이사이 간식 캐시.

가끔 신호가 약해져도 다시 연결하는 법을 안다,
우리 집 와이파이 비밀번호: 같이.

그리고 이건 무제한 요금제—
하루치 걱정은 로그아웃, 웃음은 평생 데이터 무한.

## Tools can stream too!
스트리밍은 일반적으로 최종 결과가 준비되기 전에 사용자에게 정보를 전달하는 것을 의미합니다. 이는 유용한 경우가 많습니다. `get_stream_writer` 라이터를 사용하면 생성한 소스에서 `custom` 데이터를 쉽게 스트리밍할 수 있습니다.

In [9]:
from langchain.agents import create_agent
from langgraph.config import get_stream_writer


def get_weather(city: str) -> str:
    """Get weather for a given city."""
    writer = get_stream_writer()
    # stream any arbitrary data
    writer(f"Looking up data for city: {city}")
    writer(f"Acquired data for city: {city}")
    return f"It's always sunny in {city}!"


agent = create_agent(
    model="openai:gpt-5-mini",
    tools=[get_weather],
)

for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "샌프란시스코 날씨는 어때?"}]},
    stream_mode=["values", "custom"],
):
    print(chunk)

('values', {'messages': [HumanMessage(content='샌프란시스코 날씨는 어때?', additional_kwargs={}, response_metadata={}, id='2c5701dc-3027-47a3-88c0-f40c7b71edb4')]})
('values', {'messages': [HumanMessage(content='샌프란시스코 날씨는 어때?', additional_kwargs={}, response_metadata={}, id='2c5701dc-3027-47a3-88c0-f40c7b71edb4'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 88, 'prompt_tokens': 138, 'total_tokens': 226, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 64, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-Crwae4cS7WxViQIhlIqrVzluyx36w', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b67c6-ca87-75e3-97e3-ca7909e5bdbd-0', tool_calls=[{'name': 'get_weather', 'args': {'city':

In [10]:
for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "샌프란시스코 날씨는 어때?"}]},
    stream_mode=["custom"],
):
    print(chunk)

('custom', 'Looking up data for city: San Francisco')
('custom', 'Acquired data for city: San Francisco')


## Try different modes on your own!
Modify the stream mode and the select to produce different results.

In [11]:
for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode=["values", "custom"],
):
    if chunk[0] == "custom":
        print(chunk[1])

Looking up data for city: San Francisco, CA
Acquired data for city: San Francisco, CA
